# TalkingData AdTracking 데이터의 탐색적 분석 (Exploratory data analysis)
### 0. 데이터 필드 정보 및 의견
  - `ip`: ip address of click
  - `app`: app id for marketing
  - `device`: device type id of user mobile phone (e.g., iphone 6 plus, iphone 7, huawei mate 7, etc.)
  - `os`: os version id of user mobile phone
  - `channel`: channel id of mobile ad publisher
  - `click_time`: timestamp of click (UTC)
  - `attributed_time`: if user download the app for after clicking an ad, this is the time of the app download
    * 데이터 필드 중 유일하게 attributed_time 은 학습 데이터에만 존재하는데 다른 필드 데이터들로 하여금 테스트 데이터 내에 이 필드값의 예측치를 채워넣을 수 있을지 모르니  **반드시 확인하는 작업을 진행할 계획**입니다. 
  - `is_attributed`: the target that is to be predicted, indicating the app was downloaded

### 1. Upvote 1위의 EDA 자료를 요약 (by anokas)
- 자료 링크 - https://www.kaggle.com/anokas/talkingdata-adtracking-eda
  
- 예측 확률에 따라 평가되는 LogLoss 방식과는 달리 0, 1을 확실히 구분해내는 것이 중요한 AUROC 평가에 해당합니다.
    > 확률의 리스케일링 (상수 및 승수의 추가) 는 스코어에 영향이 없습니다.
  
- 압축된 1.8Gb 의 학습 데이터는 압축을 해제하면 무려 7.5Gb
    ```bash
    # File sizes
    train.csv                     7537.65MB
    test.csv                      863.27MB
    sample_submission.csv         195.58MB
    train_sample.csv              4.08MB

    # Line count:
    184903891 ../input/train.csv
    18790470 ../input/test.csv
    100001 ../input/train_sample.csv

    ```
    
- 저자는 백만개 라인만 입력으로 사용하였습니다.
- 데이터값이 전부 인코딩되어 있어서 다양한 차트를 통해 얻을 수 있는 인사이트가 **제한**됩니다.
- 그러므로, 저자는 각 필드의 인코딩값의 유니크수, 인코딩 값별 빈도수의 분포도, 빈도수별 ID수를 나타내는 히스토그램 세가지 차트를 확인하였습니다.
    * 약 4만 정도 index 근처 몇개 ip 들이 월등히 높은 6000 이상의 클릭수를 보이고 있습니다.
    * app 은 명확하진 않지만 index 180 근처에 조금은 집중된 모습입니다.
    * device 는 index 290 근처에 매우 높은 클릭수를 보인 1개 정도의 device 값이 있는 것이 확인됩니다.
    * os 도 index 160 근처에서 높은 클릭수를 보인 2개 정도의 os 값이 확인됩니다.
- 위 차트에서 확인한 것처럼 매우 극히 일부분의 필드값에 대해서만 Fraudulent 판정이 된 것으로 예상할 수 있습니다.
    * 백만개 라인 중 겨우 0.17% 가 Fraudulent 라고 판정된 데이터로 **결론적으로 매우 불균형적으로 학습될 수 있는 데이터이므로 over 또는 under 샘플링이 반드시 필요할 것으로 보입니다.**
    * **또는, 적절한 뉴럴넷의 하이퍼 파라미터 조정을 통해 이를 극복하거나 내부적으로 적절히 샘플링에 대응하는 Boolean 하이퍼 파라미터들을 포함하는 XGBoost, LightGBM 을 적용해보는 것이 좋은 방향 같습니다.**

### 2. EDA 체험

In [6]:
%%html
<!-- 에디터 폰트를 조정합니다. -->
<style type='text/css'>
.CodeMirror{
    font-size: 14px;
    font-family: D2Coding;
</style>

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
print(os.listdir("./data"))

['sample_submission.csv', 'test.csv', 'train.csv', 'train_sample.csv']


In [10]:
# make wider graphs
sns.set(rc={'figure.figsize':(12,8)});
plt.figure(figsize=(12,8));

<Figure size 864x576 with 0 Axes>

In [ ]:
# import first 10,000,000 rows of train and all test data
train = pd.read_csv('./data/train.csv', nrows=10000000)
test = pd.read_csv('./data/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

describe 함수를 사용하여 category, datetime 측면의 정보를 얻기위해 type 을 설정합니다.

In [ ]:
categorical_fields = ['ip', 'app', 'device', 'os', 'channel']
for field in categorical_fields:
    train[field] = train[field].astype('category')
    test[field]=test[field].astype('category')
train['is_attributed']=train['is_attributed'].astype('category')
    
# set click_time and attributed_time as timeseries
train['click_time'] = pd.to_datetime(train['click_time'])
train['attributed_time'] = pd.to_datetime(train['attributed_time'])
test['click_time'] = pd.to_datetime(test['click_time'])

top 는 most common value 를 의미합니다. 그리고 그 반복횟수가 freq 입니다.

In [ ]:
train.describe()

필드별 유니크한 값 카운트입니다. ip 필드가 특별히 많은 유니크수를 보이고 있어서,
이들이 잘 분류(그룹)시킬 수 있다면 학습 데이터의 차원(Dimension)수를 줄여 모델 구축 속도를 개선함으로써 더 큰 볼륨의 학습 데이터를 가지고 모델을 만들어 볼 수 있을 것이라고 생각해보았습니다.

In [ ]:
cols = ['ip', 'app', 'device', 'os', 'channel']
uniques = [len(train[col].unique()) for col in cols]

sns.set(font_scale=2)
pal = sns.color_palette()
ax = sns.barplot(cols, uniques, palette=pal, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')

for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

클릭 시간의 흐름대로 fradulent conversion 추이를 확인해보았습니다.

In [ ]:
df_fradulent_conversion = train[["click_time", "attributed_time"]].query('attributed_time != "NaT"')
df_fradulent_conversion["click_time"] = df_fradulent_conversion["click_time"].astype(int)
df_fradulent_conversion["attributed_time"] = df_fradulent_conversion["attributed_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
df_fradulent_conversion.head(10)
#sns.jointplot(x="click_time", y="attributed_time", data=df_fradulent_conversion, kind='scatter')

Not Fradulent 클래스의 데이터가 대부분임에 주의해야합니다.

In [ ]:
plt.figure(figsize=(8, 8))
sns.set(font_scale=1.2)
mean = (train.is_attributed.values == 1).mean()
ax = sns.barplot(['Fraudulent (1)', 'Not Fradulent (0)'], [mean, 1-mean], palette=pal)
ax.set(xlabel='Target Value', ylabel='Probability', title='Target value distribution')
for p, uniq in zip(ax.patches, [mean, 1-mean]):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height+0.01,
            '{}%'.format(round(uniq * 100, 2)),
            ha="center") 

Fradulent 에 해당하는 데이터를 오버샘플링할때 ip 특정 range 에 가중치를 부여해볼 수 있을 것 같았지만, 시간대별 샘플 수를 보니 의미없다는 것을 확인하였습니다.

In [ ]:
fraudip = train[["ip", "is_attributed"]][train["is_attributed"]==1]["ip"]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.hist(fraudip, bins=100)
plt.title("Histogram of counts for {}".format('fraudip'))
plt.ylabel('count')
plt.xlabel('fraudip')
plt.show()

In [ ]:
click_time = train["click_time"]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.hist(fraudip, bins=1000)
plt.title("Histogram of counts for {}".format('click_time'))
plt.ylabel('click_time')
plt.xlabel('count')
plt.show()

In [ ]:
train['is_attributed']=train['is_attributed'].astype(int)
train[['ip', 'is_attributed']].groupby('ip', as_index=False).mean().sort_values('is_attributed', ascending=False)